In [1]:
import branca
import geopandas as gpd
from calitp_data_analysis import gcs_geopandas, geography_utils
from rt_analysis import signal_tools
from shared_utils import catalog_utils, rt_dates, rt_utils, webmap_utils

In [2]:
# constants
TARGET_DATE = rt_dates.DATES["jul2025"]
SIGNAL_URI = "gs://calitp-analytics-data/data-analyses/rt_delay/signals/signals_2025-09-08.geojson"
TARGET_TIME_OF_DAY = "AM Peak"
ANALYSIS_DISTRICT_NUMBER = 7
TARGET_TIME_OF_DAY_LENGTH_HOURS = (
    3  # the length of the target time of day (3 hours for am peak)
)

In [3]:
g = gcs_geopandas.GCSGeoPandas()

In [4]:
# read geo files
shared_data_catalog = catalog_utils.get_catalog("shared_data_catalog")
gtfs_data_constants = catalog_utils.get_catalog("gtfs_analytics_data")

# Get district polygons to mask
districts = shared_data_catalog.caltrans_districts.read()
analysis_district = districts.loc[districts["DISTRICT"] == ANALYSIS_DISTRICT_NUMBER]
# analysis_district = gpd.read_file("central_westside.geojson")

# Get speedmap data
speedmap_segments = g.read_parquet(
    f"{gtfs_data_constants.speedmap_segments.dir}{gtfs_data_constants.speedmap_segments.segment_timeofday}_{TARGET_DATE}.parquet",
    filters=[
        ("time_of_day", "=", TARGET_TIME_OF_DAY)
    ],  # Filter for only a selected time of day
).clip(analysis_district)
# Get signal data
signals = (
    g.read_file(
        SIGNAL_URI,
    )
    .rename(columns=lambda s: s.lower())
    .clip(analysis_district)
)  # we want columns to be all lower case

# Get signal data
# Filter out devices that aren't actually signals
signals_with_transit = signals.loc[signals["tms_unit_type"] == "Traffic Signals"].set_index("objectid")

In [5]:
# Get one GDF with signals and their nearest segment
buffered_speedmap_segments = gpd.GeoDataFrame(
    data=speedmap_segments.drop(speedmap_segments.geometry.name, axis=1),
    geometry=speedmap_segments.to_crs(geography_utils.CA_NAD83Albers_m).buffer(5),
)

In [6]:
def fix_multiline_strings(geom: gpd.GeoSeries) -> gpd.GeoSeries:
    """Replace any MultiLineStrings in the input with LineStrings, keeping only the longest length"""
    # Filter out linestrings
    copy = gpd.GeoDataFrame(geometry=geom.loc[geom.geom_type == "MultiLineString"])

    # Get a unique value (in case the index is weird)
    copy["unique_value"] = 1
    copy["unique_value"] = copy["unique_value"].cumsum()

    # Explode the MultiLineStrings into their constituent linestrings
    exploded = copy.explode(copy.geometry.name, index_parts=False)

    # Pick out the line strings with the longest length
    exploded["geom_length"] = exploded.to_crs(geography_utils.CA_NAD83Albers_m).length
    exploded_sorted = exploded.sort_values(
        ["unique_value", "geom_length"], ascending=False
    )
    condensed = exploded_sorted.drop_duplicates(
        subset=["unique_value"], keep="first"
    ).drop("unique_value", axis=1)

    # Combine the new geometry with the original geometry, and return it
    return geom.where(geom.geom_type != "MultiLineString", condensed.geometry)


unique_segment_identifiers = ["segment_id", "shape_id"]

In [7]:
# Join segments to signals

sjoined_signals_segments = (
    signal_tools.sjoin_signals(
        signal_gdf=signals_with_transit.reset_index(),
        segments_gdf=buffered_speedmap_segments,
        segments_lines_gdf=speedmap_segments,
    )
    .drop("geometry", axis=1)
    .set_geometry("line_geom")
)
"""sjoined_signals_segments.geometry = fix_multiline_strings(
    sjoined_signals_segments.geometry
)
# Get whether the signal is "approaching" or not
sjoined_signals_segments["approaching"] = signal_tools.determine_approaching(
    sjoined_signals_segments
)  # unsure if this works"""
# Get the distance from the segment to the associate signal
sjoined_signals_segments["distance_to_signal"] = (
    sjoined_signals_segments["line_geom"]
    .to_crs(geography_utils.CA_NAD83Albers_m)
    .distance(
        sjoined_signals_segments["signal_pt_geom"].to_crs(
            geography_utils.CA_NAD83Albers_m
        )
    )
)
"""# Get the number of scheduled vehicles per hour
sjoined_signals_segments["vehicles_per_hour_sch"] = (
    sjoined_signals_segments["n_trips_sch"] / TARGET_TIME_OF_DAY_LENGTH_HOURS
)"""

# Filter for only approaching vehicles
"""sjoined_signals_segments_approaching = sjoined_signals_segments.loc[
    sjoined_signals_segments["approaching"]
].copy()"""

'sjoined_signals_segments_approaching = sjoined_signals_segments.loc[\n    sjoined_signals_segments["approaching"]\n].copy()'

In [8]:
sjoined_signals_segments.columns

Index(['schedule_gtfs_dataset_key', 'shape_id', 'shape_array_key', 'route_id',
       'direction_id', 'stop_pair', 'segment_id', 'stop_pair_name',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph',
       'n_trips_sch', 'trips_hr_sch', 'route_short_name', 'name',
       'caltrans_district', 'organization_source_record_id_x',
       'organization_name', 'base64_url', 'imms_id_x', 'objectid', 'location',
       'imms_id_y', 'signal_pt_geom', 'line_geom',
       'organization_source_record_id_y', 'distance_to_signal'],
      dtype='object')

In [9]:
sjoined_signals_segments.loc[sjoined_signals_segments.imms_id_y == "07LA101 -ED341"].set_geometry("signal_pt_geom").objectid

8444    11678
8446    11678
8448    11678
8450    11678
8452    11678
8454    11678
8456    11678
8458    11678
8461    11678
Name: objectid, dtype: int64

In [10]:
sjoined_signals_segments.loc[sjoined_signals_segments.objectid == 11678]

,schedule_gtfs_dataset_key,shape_id,shape_array_key,route_id,direction_id,stop_pair,segment_id,stop_pair_name,time_of_day,p50_mph,...,organization_name,base64_url,imms_id_x,objectid,location,imms_id_y,signal_pt_geom,line_geom,organization_source_record_id_y,distance_to_signal
8444,2a0571758141f412b6a546fd70a65bf3,802EB_190513,4e547674c67b66dc9a0aea1afa3dad9f,802,0.0,80204__80205,80204-80205-1,Hollywood / Vine Station__Hollywood / Western ...,AM Peak,27.45,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaW...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.32620 34.10157, -118.32276 34...",recPnGkwdpnr8jmHB,8.582111
8446,2a0571758141f412b6a546fd70a65bf3,802EB_190513,4e547674c67b66dc9a0aea1afa3dad9f,802,0.0,80204__80205,80204-80205-2,Hollywood / Vine Station__Hollywood / Western ...,AM Peak,28.54,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaW...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.31537 34.10164, -118.31431 34...",recPnGkwdpnr8jmHB,97.607837
8448,0666caf3ec1ecc96b74f4477ee4bc939,2170345_JUNE25,05eb828ee0a82c8c762f25fe271a3abe,217-13188,0.0,2477__2495,2477-2495-1,Hollywood / Bronson__Hollywood / Wilton,AM Peak,10.74,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.31816 34.10169, -118.31800 34...",recPnGkwdpnr8jmHB,15.183975
8450,0666caf3ec1ecc96b74f4477ee4bc939,1800274_JUNE25,54eb5bf91b5d77665112025af7b2aaf2,180-13188,0.0,2477__2495,2477-2495-1,Hollywood / Bronson__Hollywood / Wilton,AM Peak,9.80,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.31816 34.10169, -118.31800 34...",recPnGkwdpnr8jmHB,15.183975
8452,0666caf3ec1ecc96b74f4477ee4bc939,1800277_JUNE25,43eaf4d38ef2a9b30a0efa6df67c3309,180-13188,1.0,11030__11011,11030-11011-1,Hollywood / Wilton__Hollywood / Bronson,AM Peak,14.63,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.31346 34.10171, -118.31363 34...",recPnGkwdpnr8jmHB,15.183975
8454,0666caf3ec1ecc96b74f4477ee4bc939,2170342_JUNE25,4a8b9b32acfa2dc7d6a4796a1f45ee09,217-13188,1.0,11030__11011,11030-11011-1,Hollywood / Wilton__Hollywood / Bronson,AM Peak,12.99,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.31346 34.10171, -118.31363 34...",recPnGkwdpnr8jmHB,15.183975
8456,2a0571758141f412b6a546fd70a65bf3,802WB_190513,a4503fb729d2e45ca75ea85c7a471865,802,1.0,80205__80204,80205-80204-1,Hollywood / Western Station__Hollywood / Vine ...,AM Peak,29.06,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX3JhaW...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.30859 34.10179, -118.31123 34...",recPnGkwdpnr8jmHB,20.371752
8458,0666caf3ec1ecc96b74f4477ee4bc939,2070303_JUNE25,f91aefb6d68064f536c009b3ad2117c2,207-13188,0.0,11030__2088,11030-2088-1,Hollywood / Wilton__Franklin / Beachwood,AM Peak,13.64,...,Los Angeles County Metropolitan Transportation...,aHR0cHM6Ly9naXRsYWIuY29tL0xBQ01UQS9ndGZzX2J1cy...,07LA101 -ED341,11678,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",07LA101 -ED341,POINT (155329.119 -433522.105),"LINESTRING (-118.31346 34.10171, -118.31363 34...",recPnGkwdpnr8jmHB,28

In [11]:
# Make sure we only count the one shape per signal
signals_segments_removed_duplicates = sjoined_signals_segments.sort_values(
    ["distance_to_signal"], ascending=True
).drop_duplicates(subset=["shape_id", "objectid"], keep="first")

# Groupby signal id
speedmaps_grouped_by_signal = signals_segments_removed_duplicates.groupby("objectid")

# Get frequencies through a stop
signals_with_transit["trips_hr_sch"] = speedmaps_grouped_by_signal[
    "trips_hr_sch"
].sum()
# Get all the routes that serve a stop
agg_names = lambda names: ", ".join(names.drop_duplicates().dropna())
signals_with_transit["route_names_aggregated"] = (
    speedmaps_grouped_by_signal["route_short_name"]
).agg(agg_names)
signals_with_transit["organization_names_aggregated"] = (
    speedmaps_grouped_by_signal["organization_name"]
).agg(agg_names)

In [12]:
signals_with_transit.loc[signals_with_transit.imms_id == "07LA101 -ED341"]

,tms_unit_type,asset_sub_type,district,county,route,prefix,postmile,suffix,direction,location,...,final_cca,final_cca_source,ecwc,condition_at_end_of_2027,status_at_end_of_2027,data_date,geometry,trips_hr_sch,route_names_aggregated,organization_names_aggregated
objectid,,,,,,,,,,,,,,,,,,,,,
11678,Traffic Signals,None,7,LA,101,None,6.51,None,None,"NB,HOLLYWOOD BLVD. -VAN NESS AVE",...,NaN,None,None,Good,Good,1743033600000,POINT (-118.31642 34.10156),44.332,"217, 180, 207",Los Angeles County Metropolitan Transportation...


In [13]:
sjoined_signals_segments.columns

Index(['schedule_gtfs_dataset_key', 'shape_id', 'shape_array_key', 'route_id',
       'direction_id', 'stop_pair', 'segment_id', 'stop_pair_name',
       'time_of_day', 'p50_mph', 'n_trips', 'p20_mph', 'p80_mph',
       'n_trips_sch', 'trips_hr_sch', 'route_short_name', 'name',
       'caltrans_district', 'organization_source_record_id_x',
       'organization_name', 'base64_url', 'imms_id_x', 'objectid', 'location',
       'imms_id_y', 'signal_pt_geom', 'line_geom',
       'organization_source_record_id_y', 'distance_to_signal'],
      dtype='object')

In [14]:
# Get GDFs formatted for display on the webmap

# Segment GDF
arrowized_gdf = (
    sjoined_signals_segments.drop(["signal_pt_geom"], axis=1)
    .to_crs(geography_utils.CA_NAD83Albers_m)
    .rename_geometry("geometry")
    #.rename(columns={"organization_name": "segment_type"})
)
arrowized_gdf.geometry = arrowized_gdf.geometry.apply(rt_utils.try_parallel)
arrowized_gdf = arrowized_gdf.apply(
    rt_utils.arrowize_by_frequency, axis=1, frequency_col="trips_hr_sch"
)
arrowized_gdf["route_short_name"] = arrowized_gdf["organization_name"] + " - " + arrowized_gdf["route_short_name"]

# Signal GDF
buffered_signals = gpd.GeoDataFrame(
    data=signals_with_transit,
    geometry=signals_with_transit.to_crs(geography_utils.CA_NAD83Albers_m).buffer(50),
).reset_index()

In [15]:
arrowized_gdf.loc[1, "route_short_name"]

'Long Beach Transit - 171'

In [16]:
# Define columns to include
signals_with_transit_display_columns = [
    "tms_unit_type",
    "asset_sub_type",
    # "tms_id",
    "imms_id",
    "delegation_type",
    # "leased_owned",
    # "comment",
    "trips_hr_sch",
    "route_names_aggregated",
    "organization_names_aggregated",
    signals_with_transit.geometry.name,
]
arrowized_segments_display_columns = [
    "trips_hr_sch",
    "p50_mph",
    "p20_mph",
    "p80_mph",
    "route_short_name",
    "stop_pair_name",
    "segment_id",
    "route_id",
    "shape_id",
    arrowized_gdf.geometry.name,
]

In [17]:
arrowized_gdf[arrowized_segments_display_columns]

,trips_hr_sch,p50_mph,p20_mph,p80_mph,route_short_name,stop_pair_name,segment_id,route_id,shape_id,geometry
0,0.333,9.98,9.98,9.98,Long Beach Transit - 121,2nd & E Campo SW__PCH & 2nd NE,0658-1465-1,121,1210166,"POLYGON ((174335.977 -471490.112, 174402.060 -..."
1,2.000,22.87,18.11,34.53,Long Beach Transit - 171,PCH & Marketplace E__PCH & 2nd NE,4037-1465-1,171,1710191,"POLYGON ((175210.091 -471399.104, 175180.772 -..."
2,2.000,22.87,18.11,34.53,Long Beach Transit - 171,PCH & Marketplace E__PCH & 2nd NE,4037-1465-1,171,1710191,"POLYGON ((175210.091 -471399.104, 175180.772 -..."
3,1.000,16.62,11.09,21.52,Orange County Transportation Authority - 1,PACIFIC COAST-WESTMINSTER__PACIFIC COAST-WESTM...,4070-4071-1,1,0100,"POLYGON ((175201.872 -471407.120, 175179.605 -..."
4,1.000,16.62,11.09,21.52,Orange County Transportation Authority - 1,PACIFIC COAST-WESTMINSTER__PACIFIC COAST-WESTM...,4070-4071-1,1,0100,"POLYGON ((175201.872 -471407.120, 175179.605 -..."
...,...,...,...,...,...,...,...,...,...,...
40144,1.000,12.79,7.40,17.54,City of Simi Valley - Route 16,HWY 150 & HWY 33__Ojai Ave & Blanche,3285298-3285299-2,3404,11066,"POLYGON ((68946.401 -396271.717, 68947.555 -39..."
40145,1.000,12.79,7.40,17.54,City of Simi Valley - Route 16,HWY 150 & HWY 33__Ojai Ave & Blanche,3285298-3285299-2,3404,11066,"POLYGON ((68946.401 -396271.717, 68947.555 -39..."
40146,1.000,12.79,7.40,17.54,City of Simi Valley - Route 16,HWY 150 & HWY 33__Ojai Ave & Blanche,3285298-3285299-2,3404,11066,"POLYGON ((68946.401 -396271.717, 68947.555 -39..."
40147,1.000,12.79,7.40,17.54,City of Simi Valley - Route 16,HWY 150 & HWY 33__Ojai Ave & Blanche,3285298-3285299-2,3404,11066,"POLYGON ((68946.401 -396271.717, 68947.555 -39..."


In [18]:
study_corridors = gpd.read_file("study_corridors_lines.geojson")
study_corridors.geometry = study_corridors.to_crs(geography_utils.CA_NAD83Albers_m).buffer(100, cap_style="flat")

In [19]:
# Create webmap
SIGNAL_LEGEND_URL = "https://storage.googleapis.com/calitp-map-tiles/signal_legend.svg"
signal_colorscale = branca.colormap.step.Purples_05.scale(
    vmin=0, vmax=sjoined_signals_segments["trips_hr_sch"].max()
)
signal_folder = "signals_v12_22/"
study_corridor_map = webmap_utils.set_state_export(
    study_corridors,
    subfolder=signal_folder,
    filename="study_corridors",
    #cmap=study_colormap,
    #color_col="color_number",
    map_title="Study Corridors"
)
speedmap = webmap_utils.set_state_export(
    arrowized_gdf[arrowized_segments_display_columns],
    subfolder=signal_folder,
    filename="speeds",
    cmap=rt_utils.ACCESS_ZERO_THIRTY_COLORSCALE,
    color_col="p20_mph",
    cache_seconds=1,
    map_type="new_speedmap",
    legend_url=rt_utils.SPEEDMAP_LEGEND_URL,
    map_title="Speeds",
    existing_state=study_corridor_map["state_dict"],
)
signal_speedmap = webmap_utils.set_state_export(
    buffered_signals[signals_with_transit_display_columns],
    subfolder=signal_folder,
    cmap=signal_colorscale,
    color_col="trips_hr_sch",
    existing_state=speedmap["state_dict"],
    map_title=f"Signals with Approach Speeds {TARGET_DATE}",
    #legend_url=SIGNAL_LEGEND_URL,
    manual_centroid=[34.048108, -118.4183252],
)
signal_speedmap

/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())
/home/jovyan/data-analyses/_shared_utils/shared_utils/webmap_utils.py:118: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = (gdf.geometry.centroid.y.mean(), gdf.geometry.centroid.x.mean())


{'state_dict': {'name': 'null',
  'layers': [{'name': 'Study Corridors',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_22/study_corridors.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}},
   {'name': 'Speeds',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_22/speeds.geojson.gz',
    'properties': {'stroked': False,
     'highlight_saturation_multiplier': 0.5,
     'tooltip_speed_key': 'p20_mph'},
    'type': 'new_speedmap'},
   {'name': 'Signals with Approach Speeds 2025-07-16',
    'url': 'https://storage.googleapis.com/calitp-map-tiles/signals_v12_22/test2.geojson.gz',
    'properties': {'stroked': False, 'highlight_saturation_multiplier': 0.5}}],
  'lat_lon': [34.048108, -118.4183252],
  'zoom': 13,
  'legend_url': 'https://storage.googleapis.com/calitp-map-tiles/speeds_legend.svg'},
 'spa_link': 'https://embeddable-maps.calitp.org/?state=eyJuYW1lIjogIm51bGwiLCAibGF5ZXJzIjogW3sibmFtZSI6ICJTd

In [24]:
# Get signal-route grain data
signal_route_group = signals_segments_removed_duplicates.groupby(
    ["objectid", "route_short_name", "organization_name", "direction_id"]
)

signals_routes_frequency = signal_route_group["trips_hr_sch"].sum()
merged_signals_routes_frequency = signals_routes_frequency.reset_index().merge(
    signals_with_transit[
        [
            "tms_unit_type",
            "asset_sub_type",
            "location",
            "tms_id",
            "imms_id",
            "delegation_type",
            "leased_owned",
            "comment",
            "geometry",
        ]
    ],
    how="left",
    left_on="objectid",
    right_index=True,
    validate="many_to_one",
)
merged_geometry = gpd.GeoSeries(merged_signals_routes_frequency["geometry"]).centroid.to_crs(geography_utils.WGS84)
merged_signals_routes_frequency["latitude"] = merged_geometry.y.round(5)
merged_signals_routes_frequency["longitude"] = merged_geometry.x.round(5)
merged_signals_routes_frequency.drop("geometry", axis=1).to_csv("signals_routes_table.csv", index=False)

In [23]:
merged_signals_routes_frequency

,objectid,route_short_name,organization_name,direction_id,trips_hr_sch,tms_unit_type,asset_sub_type,location,tms_id,imms_id,delegation_type,leased_owned,comment,geometry,latitude,longitude
0,11083,1,Orange County Transportation Authority,0.0,1.667,Traffic Signals,None,STUDEBAKER RD.,07LA001-0.210,07LA001 -ED020,Category 4 Signals – Owned & operated by state...,CT Owned,approximated TRD based on controller data.\n\n...,"POLYGON ((175453.299 -471720.409, 175453.058 -...",33.753808,-118.107158
1,11083,1,Orange County Transportation Authority,1.0,1.000,Traffic Signals,None,STUDEBAKER RD.,07LA001-0.210,07LA001 -ED020,Category 4 Signals – Owned & operated by state...,CT Owned,approximated TRD based on controller data.\n\n...,"POLYGON ((175453.299 -471720.409, 175453.058 -...",33.753808,-118.107158
2,11083,131,Long Beach Transit,0.0,2.000,Traffic Signals,None,STUDEBAKER RD.,07LA001-0.210,07LA001 -ED020,Category 4 Signals – Owned & operated by state...,CT Owned,approximated TRD based on controller data.\n\n...,"POLYGON ((175453.299 -471720.409, 175453.058 -...",33.753808,-118.107158
3,11083,131,Long Beach Transit,1.0,2.000,Traffic Signals,None,STUDEBAKER RD.,07LA001-0.210,07LA001 -ED020,Category 4 Signals – Owned & operated by state...,CT Owned,approximated TRD based on controller data.\n\n...,"POLYGON ((175453.299 -471720.409, 175453.058 -...",33.753808,-118.107158
4,11083,171,Long Beach Transit,0.0,2.000,Traffic Signals,None,STUDEBAKER RD.,07LA001-0.210,07LA001 -ED020,Category 4 Signals – Owned & operated by state...,CT Owned,approximated TRD based on controller data.\n\n...,"POLYGON ((175453.299 -471720.409, 175453.058 -...",33.753808,-118.107158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4154,20134,123,Orange County Transportation Authority,1.0,1.000,Traffic Signals,None,5 N/B Off Ramp at Artesia Blvd / Firestone Blvd,12-Signals-555-Art,12ORA005 -E0576,"Category 1 Signals – Owned, operated & maintai...",CT Owned,3/22/17 Upgraded to 2070 controller\n3...,"POLYGON ((184026.066 -458253.841, 184025.825 -...",33.873631,-118.011625
4155,20208,29,Orange County Transportation Authority,0.0,2.333,Traffic Signals,None,Beach Blvd at Rosecrans,12-Signals-644-Rose,12ORA039 -E0293,"Category 1 Signals – Owned, operated & maintai...",CT Owned,approximated TRD based on controller data,"POLYGON ((186247.132 -455669.477, 186246.892 -...",33.896505,-117.987034
4156,20208,29,Orange County Transportation Authority,1.0,2.333,Traffic Signals,None,Beach Blvd at Rosecrans,12-Signals-644-Rose,12ORA039 -E0293,"Category 1 Signals – Owned, operated & maintai...",CT Owned,approximated TRD based on controller data,"POLYGON ((186247.132 -455669.477, 186246.892 -...",33.896505,-117.987034
4157,20210,29,Orange County Transportation Authority,0.0,2.333,Traffic Signals,None,Beach Blvd at Hillsborough Dr.,12-Signals-646-Hill,12ORA039 -E0049,"Category 1 Signals – Owned, operated & maintai...",CT Owned,approximated TRD based on controller data,"POLYGON ((187039.364 -454321.515, 187039.123 -...",33.908504,-117.978161
